In [ ]:
import sys
sys.path.append("fwks_lib")

# limit the memory usage
import keras.backend as K
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [ ]:
from fwks.tasks import FeatureLearnabilityTask, Task
from dissertation_model import (
    mfcc_model,
    handcrafted_model,
    plc_model,
    czt_model
)


import keras

FeatureLearnabilityTask.how_much = 3000 # 9k causes memory error


def small_network(self, source_shape, target_shape):
    source_shape = list(source_shape[1:])
    source_shape[0] = None
    mapping = self.get_mapping()
    mdl = keras.models.Sequential()
    mdl.add(keras.layers.Dense(512, activation='relu', input_shape=source_shape))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(target_shape[-1]))
    return mdl


def small_reg_network(self, source_shape, target_shape):
    source_shape = list(source_shape[1:])
    source_shape[0] = None
    mapping = self.get_mapping()
    mdl = keras.models.Sequential()
    mdl.add(keras.layers.Dense(512, activation='relu',
                               kernel_regularizer=keras.regularizers.L1L2(),
                               bias_regularizer=keras.regularizers.L1L2(),
                               input_shape=source_shape))
    mdl.add(keras.layers.Dense(512, activation='relu',
                               kernel_regularizer=keras.regularizers.L1L2(),
                               bias_regularizer=keras.regularizers.L1L2()))
    mdl.add(keras.layers.Dense(target_shape[-1]))
    return mdl
    
    
def large_network(self, source_shape, target_shape):
    source_shape = list(source_shape[1:])
    source_shape[0] = None
    mapping = self.get_mapping()
    mdl = keras.models.Sequential()
    mdl.add(keras.layers.Dense(512, activation='relu', input_shape=source_shape))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(target_shape[-1]))
    return mdl
    
for mapper_cb in [small_network, large_network, small_reg_network]:

    class MFCCLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return mfcc_model()

        get_mapper_network = classmethod(mapper_cb)


    class PLCLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return plc_model()

        get_mapper_network = classmethod(mapper_cb)


    class CZTLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return czt_model()

        get_mapper_network = classmethod(mapper_cb)


    class HandcraftedLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return handcrafted_model()

        get_mapper_network = classmethod(mapper_cb)

    for task in Task.all():
        print(task, mapper_cb.__name__)
        task.run("/tmp/cache")
